In [9]:
"""
Problem 1

part b

"""
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Given Data
np.random.seed()

mTrain = 100  # size of training data
mTest = 200   # size of test data
K = 5         # number of categories
n = 20        # number of features

A_true = np.random.normal(size=(K, n))
b_true = np.random.normal(size=(K, 1))
v = np.random.normal(size=(K, mTrain + mTest))  # noise
data = np.random.normal(size=(n, mTrain + mTest))
label = np.argmax(A_true @ data + np.tile(b_true, (1, mTrain + mTest)) + v, axis=0)

# Training data
x = data[:, :mTrain]
y = label[:mTrain]
# Test data
xtest = data[:, mTrain:]
ytest = label[mTrain:]


mus = np.logspace(-1, 2, num=10)
#print(mus)
errorTrain = []
errorTest = []

for mu in mus:
    A = cp.Variable((K, n))
    b = cp.Variable((K,1))
    f = A @ x + b #* np.ones((1,mTrain))

    L = 0 # cost function
    for k in range(1, K+1):
        index = np.concatenate((np.arange(k-1), np.arange(k, K)))
        L += cp.sum(cp.pos(1 + cp.max(f[index][:, np.where(y == k)[0]], axis=0) - f[k-1, y == k]))
        # where cp.pos = max{x,0}
    norm = cp.square(cp.norm(A, "fro"))
    objective = cp.Minimize(L + mu * norm)
    constraints = [cp.sum(b) == 0]
    
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # training error
    y_predTrain = np.argmax((A @ x + b.value, axis=0))
    errorTrain.append(np.sum(y_predTrain != y) / mTrain)

    # test error
    y_predTest = np.argmax((A @ xtest + b.value, axis=0))
    errorTest.append(np.sum(y_predTest != ytest) / mTest)

# plots
plt.clf()
plt.semilogx(mus, errorTrain, label='Training Error')
plt.semilogx(mus, errorTest, 'r', label='Test Error')
plt.xlabel('mu')
plt.ylabel('error')
plt.legend()
plt.show()

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3631218152.py, line 59)